In [ ]:
import pupil_classv2 as pc
import math
from statistics import mean
import csv
from timestamps import new_timestamps
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from helper import *
import tdt
from scipy.signal import find_peaks, peak_widths,resample,correlate, correlation_lags,peak_prominences,resample,butter, lfilter, freqz,medfilt,hilbert
from scipy.stats import pearsonr,wilcoxon,mannwhitneyu,kruskal
import os
from fp_helper import *
from matplotlib import rcParams
import cv2
from scipy.stats import ttest_ind,ttest_rel,zscore
from sklearn.linear_model import Ridge,RidgeCV
from sklearn.model_selection import KFold,cross_val_score
from random import randint
from scipy.ndimage import median_filter
import scikit_posthocs as sp
import statsmodels.api as sm 
from statsmodels.formula.api import ols 


params = {
        "font.family" : "Arial",
        'pdf.fonttype' : 42,
        'axes.labelsize': 10,
        'axes.titlesize': 11,
        'axes.linewidth': 0.5,
        'xtick.labelsize':9,
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'ytick.labelsize':9,
        'axes.spines.top':False,
        'axes.spines.right':False
         }
rcParams['figure.figsize'] = 21.7,8.27
rcParams.update(params)
def format_ax(ax,xlim,ylim,xspace,yspace):
   
    ax.set_xticks(np.arange(xlim[0],xlim[1]+ xspace,xspace))
    ax.set_yticks(np.arange(ylim[0],ylim[1]+ yspace,yspace))

    ax.set(ylim=(ylim[0], ylim[1]))
    ax.set(xlim=(xlim[0], xlim[1]))

In [ ]:
###low pass filter pupil
def filter_pupil(pupil,order = 1,cutoff = 1,fs = 20):
    def butter_lowpass(cutoff, fs, order=5):
        return butter(order, cutoff, fs=fs, btype='low', analog=False)

    def butter_lowpass_filter(data, cutoff, fs, order=5):
        b, a = butter_lowpass(cutoff, fs, order=order)
        y = lfilter(b, a, data)
        return y

    return butter_lowpass_filter(pupil, cutoff, fs, order)

In [ ]:
### get pupil events for ACC
###get events from all sessions

def get_acc_events(id,date):
    id = id
    date = date
    face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
    path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
    df = pd.read_csv(path)
    pupil = np.array(df["Pupil Size"])
    time = np.array(df["Time"])
    speed = np.array(df["Running Speed"])
    speed[0] = 0
    dff = np.array(df["dFF"])
    face = np.array(pd.read_csv(face_path)["Facial Movement"])

    ###Onset detection alogrithm

    
    filter = filter_pupil(pupil) ###first apply low pass filter to pupil trace

    def groupSequence(x): 
        it = iter(x) 
        prev, res = next(it), [] 
    
        while prev is not None: 
            start = next(it, None) 
    
            if start and start > prev:

                res.append(prev) 
            elif res: 
                yield list(res + [prev]) 
                res = [] 
            prev = start 


    regions = list(groupSequence(filter))
    corrected_regions = [region for region in regions if len(region) >=15 ] ###only include regions that are >750ms
    corrected_regions = [region for region in corrected_regions if region[0]<np.median(filter) ] ###only include onsets that occur below z = median

    region_ix = [[np.where(filter == val)[0][0] for val in region] for region in corrected_regions] ### get indices for each region


    ##join regions that are near eachother
    new = []
    for i,first in enumerate(region_ix):
        if i < len(region_ix)-1:
            second = region_ix[i + 1]
            end = first[-1]
            start = second[0]
            if first[0] - region_ix[i - 1][-1]>20:
                if start-end<= 20:
                    #print(first[0]/20,second[0]/20)
                    new.append(first + second)
                else:
                    new.append(first)
    region_ix = new
    corrected_regions = [filter[region[0]:region[-1]] for region in region_ix]


    new = []
    for i,first in enumerate(region_ix):
        if i < len(region_ix)-1:
            second = region_ix[i + 1]
            end = first[-1]
            start = second[0]
            if first[0] - region_ix[i - 1][-1]>10:
                if start-end<= 10:
                    #print(first[0]/20,second[0]/20)
                    new.append(first + second)
                else:
                    new.append(first)
    region_ix = new
    corrected_regions = [filter[region[0]:region[-1]] for region in region_ix]
    onsets_ix = [region[0] for region in region_ix]


    ###get offsets and durations for pupil event
    offset_ix = []
    durations = []
    for i,(region,index) in enumerate(zip(corrected_regions,region_ix)):
        if i < len(region_ix)-1:
            start = index[-1] ###end of region
            end = region_ix[i+1][0] ### start of next region
            onset_y = region[0]
            offsets = np.where(filter[start:end] <= onset_y) ###find ix of first point that goes below onset 
        if len(offsets[0]) == 0: ###if it never goes below onset then use lowest value
            offset = start + np.argmin(filter[start:end])
        else:
            offset = start + offsets[0][0]


        offset_ix.append(offset)

        durations.append((offset-index[0])/20)

    ###handle last pupil event
    start = region_ix[-1][-1]
    end = 36000
    onset_y = corrected_regions[-1][0]
    offsets = np.where(filter[start:end] <= onset_y)
    if len(offsets[0]) == 0:
        offset = end-1
    else:
        offset = start + np.argmin(filter[start:end])

    offset_ix[-1] = offset

    durations[-1] = (offset-region_ix[-1][0])/20

    ###get pupil event amplitudes
    amplitudes = []
    amplitudes_ix = []
    for on,off in zip(onsets_ix,offset_ix):
        #print(on,off)
        amplitudes.append(max(filter[on:off])-filter[on])
        amplitudes_ix.append(on + np.argmax(filter[on:off]))

    ###get pupil event slopes
    slopes = []
    for region in corrected_regions:
        run = len(region)/20
        rise = region[-1]-region[0]
        slopes.append(rise/run)

    ###get dff data
    aucs = []
    peak_dffs = []
    mean_dffs = []
    for onset,offset in zip(onsets_ix,offset_ix):
        baseline = np.mean(dff[onset-40:onset-10])
        peak_dffs.append(max(dff[onset:offset]) - np.mean(dff[onset-30:onset]))
        mean_dffs.append(np.mean(dff[onset:offset])- np.mean(dff[onset-20:onset]))

    ###create dictionary

    events_dict = {"durations" :durations,
    "amplitudes": amplitudes,
    "slopes" : slopes,
    "onsets" : onsets_ix,
    "offsets" : offset_ix,
    "peak_ix" : amplitudes_ix,
    "ID" : [id] * len(durations),
    "Date" : [date] * len(durations),
    "peak_dff" : peak_dffs,
    "mean_dff" : mean_dffs,


    }

    return pd.DataFrame.from_dict(events_dict)


In [ ]:
### get pupil events
###get events from all sessions

def get_lc_events(id,date):
    id = id
    date = date
    path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/Nithik/ACC_Arousal_Paper/LC_data/{id}_{date}_fixed.csv"
    df = pd.read_csv(path)
    time = df["t"]
    pupil = np.array(zscore(df["pupil"])) ###zscore pupil data


    ###Onset detection alogrithm

    
    filter = filter_pupil(pupil) ###first apply low pass filter to pupil trace

    def groupSequence(x): 
        it = iter(x) 
        prev, res = next(it), [] 
    
        while prev is not None: 
            start = next(it, None) 
    
            if start and start > prev:

                res.append(prev) 
            elif res: 
                yield list(res + [prev]) 
                res = [] 
            prev = start 


    regions = list(groupSequence(filter))
    corrected_regions = [region for region in regions if len(region) >=15 ] ###only include regions that are >750ms
    corrected_regions = [region for region in corrected_regions if region[0]<np.median(filter) ] ###only include onsets that occur below z = median

    region_ix = [[np.where(filter == val)[0][0] for val in region] for region in corrected_regions] ### get indices for each region


    ##join regions that are near eachother
    new = []
    for i,first in enumerate(region_ix):
        if i < len(region_ix)-1:
            second = region_ix[i + 1]
            end = first[-1]
            start = second[0]
            if first[0] - region_ix[i - 1][-1]>20:
                if start-end<= 20:
                    #print(first[0]/20,second[0]/20)
                    new.append(first + second)
                else:
                    new.append(first)
    region_ix = new
    corrected_regions = [filter[region[0]:region[-1]] for region in region_ix]


    new = []
    for i,first in enumerate(region_ix):
        if i < len(region_ix)-1:
            second = region_ix[i + 1]
            end = first[-1]
            start = second[0]
            if first[0] - region_ix[i - 1][-1]>10:
                if start-end<= 10:
                    #print(first[0]/20,second[0]/20)
                    new.append(first + second)
                else:
                    new.append(first)
    region_ix = new
    corrected_regions = [filter[region[0]:region[-1]] for region in region_ix]
    onsets_ix = [region[0] for region in region_ix]


    ###get offsets and durations for pupil event
    offset_ix = []
    durations = []
    for i,(region,index) in enumerate(zip(corrected_regions,region_ix)):
        if i < len(region_ix)-1:
            start = index[-1] ###end of region
            end = region_ix[i+1][0] ### start of next region
            onset_y = region[0]
            offsets = np.where(filter[start:end] <= onset_y) ###find ix of first point that goes below onset 
        if len(offsets[0]) == 0: ###if it never goes below onset then use lowest value
            offset = start + np.argmin(filter[start:end])
        else:
            offset = start + offsets[0][0]


        offset_ix.append(offset)

        durations.append((offset-index[0])/20)

    ###handle last pupil event
    start = region_ix[-1][-1]
    end = len(pupil)
    onset_y = corrected_regions[-1][0]
    offsets = np.where(filter[start:end] <= onset_y)
    if len(offsets[0]) == 0:
        offset = end-1
    else:
        offset = start + np.argmin(filter[start:end])

    offset_ix[-1] = offset

    durations[-1] = (offset-region_ix[-1][0])/20

    ###get pupil event amplitudes
    amplitudes = []
    amplitudes_ix = []
    for on,off in zip(onsets_ix,offset_ix):
        #print(on,off)
        amplitudes.append(max(filter[on:off])-filter[on])
        amplitudes_ix.append(on + np.argmax(filter[on:off]))

    ###get pupil event slopes
    slopes = []
    for region in corrected_regions:
        run = len(region)/20
        rise = region[-1]-region[0]
        slopes.append(rise/run)

    # ###get dff data
    # aucs = []
    # peak_dffs = []
    # mean_dffs = []
    # for onset,offset in zip(onsets_ix,offset_ix):
    #     baseline = np.mean(dff[onset-40:onset-10])
    #     peak_dffs.append(max(dff[onset:offset]) - np.mean(dff[onset-30:onset]))
    #     mean_dffs.append(np.mean(dff[onset:offset])- np.mean(dff[onset-20:onset]))

    ###create dictionary

    events_dict = {"durations" :durations,
    "amplitudes": amplitudes,
    "slopes" : slopes,
    "onsets" : onsets_ix,
    "offsets" : offset_ix,
    "peak_ix" : amplitudes_ix,
    "ID" : [id] * len(durations),
    "Date" : [date] * len(durations),
    #"peak_dff" : peak_dffs,
    #"mean_dff" : mean_dffs,
    }
    return pd.DataFrame.from_dict(events_dict)


In [ ]:
###read in correlation data for ACC
###correlation with dff

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}
df_list = []


for id,dates in id_date.items():
    for date in dates:
      face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
      path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
      df = pd.read_csv(path)
      pupil = np.array(df["Pupil Size"])
      time = np.array(df["Time"])
      speed =zscore(np.array(df["Running Speed"]))
      dff = np.array(df["dFF"])
      face = np.array(pd.read_csv(face_path)["Facial Movement"])
      df["Facial Movement"] = face

      new_dict = {"R":pearsonr(dff,pupil)[0], "type": "P-D", "ID" : id, "Date": date,"Region":"ACC"}
      df_list.append(new_dict)
      # new_dict = {"R":pearsonr(dff,speed)[0], "type": "D-S", "ID" : id, "Date": date}
      # df_list.append(new_dict)
      new_dict = {"R":pearsonr(dff,face)[0], "type": "F-D", "ID" : id, "Date": date,"Region":"ACC"}
      df_list.append(new_dict)

acc_r_df = pd.DataFrame.from_dict(df_list)





In [ ]:
###read in correlation data for LC



id_date = {
  "LC01":["20240419"],
  "LC02":["20240416"],
  "LC03":["20240419"],
  "LC04":["20240416","20240417"],
  "LC05":["20240513"],
  "LC06":["20240513"],
}
df_list = []


for id,dates in id_date.items():
    for date in dates:
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/Nithik/ACC_Arousal_Paper/LC_data/{id}_{date}_fixed.csv"
        df = pd.read_csv(path)
        time = df["t"]
        pupil = np.array(zscore(df["pupil"])) ###zscore pupil data
        face = np.array(zscore(df["mot_whisk"]))###zscore face data
        dff = np.array(zscore(df["fp_lc"])) ###zscore dff data

        new_dict = {"R":pearsonr(dff,pupil)[0], "type": "P-D", "ID" : id, "Date": date,"Region":"LC"}
        df_list.append(new_dict)
        # new_dict = {"R":pearsonr(dff,speed)[0], "type": "D-S", "ID" : id, "Date": date}
        # df_list.append(new_dict)
        new_dict = {"R":pearsonr(dff,face)[0], "type": "F-D", "ID" : id, "Date": date,"Region":"LC"}
        df_list.append(new_dict)

lc_r_df = pd.DataFrame.from_dict(df_list)

###combine into one correlation df
r_df = pd.concat([lc_r_df,acc_r_df])

In [ ]:
###LC vs ACC face-dff correlation
plt.figure(figsize = (0.6,1.2))
g = sns.barplot(data = r_df.groupby(["ID",'type',"Region"]).mean().reset_index().query("type == 'F-D'"), x = "Region", y = "R",errorbar="se",fill = None,errwidth= 1)
sns.stripplot(data = r_df.groupby(["ID",'type',"Region"]).mean().reset_index().query("type == 'F-D'"), x = "Region", y = "R",color = "black",alpha = 0.3,s = 4,jitter = 0.1)
g.xaxis.label.set_visible(False)
g.set_yticks(np.arange(-0.2,1,0.2))
g.set(ylim=(0, 0.8))
g.set_ylabel("F-D Correlation (R)")
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/F-D_R.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###LC vs ACC face-dff correlation stats
stat_df = r_df.groupby(["ID",'type',"Region"]).mean().reset_index().query("type == 'F-D'")
group1 = stat_df[stat_df['Region']=="LC"]
group2 = stat_df[stat_df['Region']=="ACC"]
mannwhitneyu(group1["R"], group2["R"],nan_policy= "omit")

In [ ]:
###LC vs ACC pupil-dff correlation
plt.figure(figsize = (0.6,1.2))
g = sns.barplot(data = r_df.groupby(["ID",'type',"Region"]).mean().reset_index().query("type == 'P-D'"), x = "Region", y = "R",errorbar="se",fill = None,errwidth= 1)
sns.stripplot(data = r_df.groupby(["ID",'type',"Region"]).mean().reset_index().query("type == 'P-D'"), x = "Region", y = "R",color = "black",alpha = 0.3,s = 4,jitter = 0.1)
g.xaxis.label.set_visible(False)
g.set_yticks(np.arange(-0.2,1,0.2))
g.set(ylim=(0, 0.8))
g.set_ylabel("P-D Correlation (R)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/P-D_R.pdf",format ="pdf",transparent = True,bbox_inches = "tight")

In [ ]:
###LC vs ACC pupil-dff correlation stats
stat_df = r_df.groupby(["ID",'type',"Region"]).mean().reset_index().query("type == 'P-D'")
group1 = stat_df[stat_df['Region']=="LC"]
group2 = stat_df[stat_df['Region']=="ACC"]
mannwhitneyu(group1["R"], group2["R"],nan_policy= "omit")

In [ ]:
###get cross-correlation for ACC
pre = 2
post =2


id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
graph_df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_acc_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        amplitudes = events["amplitudes"]

        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = np.array(df["Running Speed"])
        speed = zscore(speed)
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = pre, post = post)
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = pre, post = post)
        dff_matrix,dff_times = trial_align(onsets,time,dff,fps = 20,pre = pre, post = post)


        corr = correlate(np.mean(dil_matrix,axis=0), np.mean(dff_matrix,axis = 0))
        lags = correlation_lags(len(np.mean(dil_matrix,axis=0)), len(np.mean(dff_matrix,axis = 0)))
        lags = lags/20
        corr /= np.max(corr)
        i = np.argmax(corr)
        delay = lags[i]
        lag_dict = {"Cross-Correlation Delay (s)" : delay,"ID": id, "date" : date,"type":"P-D","Region":"ACC"}
        df_list.append(lag_dict)

        for lag,c in zip(lags,corr):
            graph_dict = {"Lags (s)": lag,"Cross-Correlation": c,"ID": id, "date" : date,"type":"P-D"}
            graph_df_list.append(graph_dict)
        

        corr = correlate(np.mean(face_matrix,axis=0), np.mean(dff_matrix,axis = 0))
        lags = correlation_lags(len(np.mean(face_matrix,axis=0)), len(np.mean(dff_matrix,axis = 0)))
        lags = lags/20
        corr /= np.max(corr)
        i = np.argmax(corr)
        delay = lags[i]
        lag_dict = {"Cross-Correlation Delay (s)" : delay,"ID": id, "date" : date,"type":"F-D","Region":"ACC"}
        df_list.append(lag_dict)

        for lag,c in zip(lags,corr):
            graph_dict = {"Lags (s)" : lag,"Cross-Correlation": c,"ID": id, "date" : date,"type":"F-D"}
            graph_df_list.append(graph_dict)
        
acc_lag_df = pd.DataFrame.from_dict(df_list)
# graph_df = pd.DataFrame.from_dict(graph_df_list)

In [ ]:
###get cross-correlation for LC
pre = 2
post =2


id_date = {
  "LC01":["20240419"],
  "LC02":["20240416"],
  "LC03":["20240419"],
  "LC04":["20240416","20240417"],
}

df_list = []
graph_df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_lc_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        amplitudes = events["amplitudes"]

        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/Nithik/ACC_Arousal_Paper/LC_data/{id}_{date}.csv"
        df = pd.read_csv(path)
        time = df["t"]
        pupil = np.array(zscore(df["pupil"])) ###zscore pupil data
        face = np.array(zscore(df["mot_whisk"]))###zscore face data
        dff = np.array(zscore(df["fp_lc"])) ###zscore dff data

    
        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = pre, post = post)
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = pre, post = post)
        dff_matrix,dff_times = trial_align(onsets,time,dff,fps = 20,pre = pre, post = post)


        corr = correlate(np.mean(dil_matrix,axis=0), np.mean(dff_matrix,axis = 0))
        lags = correlation_lags(len(np.mean(dil_matrix,axis=0)), len(np.mean(dff_matrix,axis = 0)))
        lags = lags/20
        corr /= np.max(corr)
        i = np.argmax(corr)
        delay = lags[i]
        lag_dict = {"Cross-Correlation Delay (s)" : delay,"ID": id, "date" : date,"type":"P-D","Region":"LC"}
        df_list.append(lag_dict)

        for lag,c in zip(lags,corr):
            graph_dict = {"Lags (s)": lag,"Cross-Correlation": c,"ID": id, "date" : date,"type":"P-D"}
            graph_df_list.append(graph_dict)
        

        corr = correlate(np.mean(face_matrix,axis=0), np.mean(dff_matrix,axis = 0))
        lags = correlation_lags(len(np.mean(face_matrix,axis=0)), len(np.mean(dff_matrix,axis = 0)))
        lags = lags/20
        corr /= np.max(corr)
        i = np.argmax(corr)
        delay = lags[i]
        lag_dict = {"Cross-Correlation Delay (s)" : delay,"ID": id, "date" : date,"type":"F-D","Region":"LC"}
        df_list.append(lag_dict)

        for lag,c in zip(lags,corr):
            graph_dict = {"Lags (s)" : lag,"Cross-Correlation": c,"ID": id, "date" : date,"type":"F-D"}
            graph_df_list.append(graph_dict)
        
lc_lag_df = pd.DataFrame.from_dict(df_list)
# graph_df = pd.DataFrame.from_dict(graph_df_list)

###combine into one x-correlation df
lag_df = pd.concat([lc_lag_df,acc_lag_df])

In [ ]:
acc_lag_df

In [ ]:
###LC vs ACC face-dff lag
plt.figure(figsize = (0.6,1.2))
g = sns.barplot(data = lag_df.groupby(["ID",'type',"Region"]).mean().reset_index().query("type == 'F-D'"), x = "Region", y = "Cross-Correlation Delay (s)",errorbar="se",fill = None,errwidth= 1)
sns.stripplot(data = lag_df.groupby(["ID",'type',"Region"]).mean().reset_index().query("type == 'F-D'"), x = "Region", y = "Cross-Correlation Delay (s)",color = "black",alpha = 0.3,s = 4,jitter = 0.1)
g.xaxis.label.set_visible(False)
g.set_yticks(np.arange(-0.3,0.3,0.1))
g.set(ylim=(-0.3, 0.2))
g.set_ylabel("F-D x-corr delay (s)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/F-D_xcorr.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###LC vs ACC face-dff lag stats
stat_df = lag_df.groupby(["ID",'type',"Region"]).mean().reset_index().query("type == 'F-D'")
group1 = stat_df[stat_df['Region']=="LC"]
group2 = stat_df[stat_df['Region']=="ACC"]
ttest_ind(group1["Cross-Correlation Delay (s)"], group2["Cross-Correlation Delay (s)"],nan_policy= "omit")

In [ ]:
###LC vs ACC face-dff lag
plt.figure(figsize = (0.6,1.2))
g = sns.barplot(data = lag_df.groupby(["ID",'type',"Region"]).mean().reset_index().query("type == 'P-D'"), x = "Region", y = "Cross-Correlation Delay (s)",errorbar="se",fill = None,errwidth= 1)
sns.stripplot(data = lag_df.groupby(["ID",'type',"Region"]).mean().reset_index().query("type == 'P-D'"), x = "Region", y = "Cross-Correlation Delay (s)",color = "black",alpha = 0.3,s = 4,jitter = 0.1)
g.xaxis.label.set_visible(False)
g.set_yticks(np.arange(0,3.5,0.5))
g.set(ylim=(0, 3))
g.set_ylabel("P-D x-corr delay (s)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/P-D_xcorr.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###LC vs ACC face-dff lag stats
stat_df = lag_df.groupby(["ID",'type',"Region"]).mean().reset_index().query("type == 'P-D'")
group1 = stat_df[stat_df['Region']=="LC"]
group2 = stat_df[stat_df['Region']=="ACC"]
ttest_ind(group1["Cross-Correlation Delay (s)"], group2["Cross-Correlation Delay (s)"],nan_policy= "omit")

In [ ]:
###onset aligned ACC activity

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_acc_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        onsets_ix = events["onsets"]
  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = filter_pupil(np.array(df["Pupil Size"]))
        time = np.array(df["Time"])
        speed = zscore(np.array(df["Running Speed"]))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = 2, post = 10 )
        trial_matrix,trial_times = trial_align(onsets,time,dff,fps = 20,pre = 2, post = 10)
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = 2, post = 10)

        for j,trial in enumerate(dil_matrix):
            baseline = np.mean(pupil[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline

            for i,Time in enumerate(dil_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date}
                df_list.append(new_dict)

        for j,trial in enumerate(trial_matrix):
            baseline = np.mean(dff[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
         
            for i,Time in enumerate(trial_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date}
                df_list.append(new_dict)

        for j,trial in enumerate(face_matrix):
            baseline = np.mean(face[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
          
            for i,Time in enumerate(face_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Face","ID" : id, "Date": date}
                df_list.append(new_dict)

acc_align_df = pd.DataFrame.from_dict(df_list)


In [ ]:
heat_df

In [ ]:
###onset aligned LC activity

id_date = {
  "LC01":["20240419"],
  "LC02":["20240416"],
  "LC03":["20240419"],
  "LC04":["20240416","20240417"],
  "LC05":["20240513"],
  "LC06":["20240513"],
}



df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_lc_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        onsets_ix = events["onsets"]
  
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/Nithik/ACC_Arousal_Paper/LC_data/{id}_{date}_fixed.csv"
        df = pd.read_csv(path)
        time = df["t"]
        pupil = filter_pupil(np.array(zscore(df["pupil"]))) ###zscore pupil data
        face = np.array(zscore(df["mot_whisk"]))###zscore face data
        dff = np.array(zscore(df["fp_lc"])) ###zscore dff data

        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = 2, post = 10 )
        trial_matrix,trial_times = trial_align(onsets,time,dff,fps = 20,pre = 2, post = 10)
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = 2, post = 10)

        for j,trial in enumerate(dil_matrix):
            baseline = np.mean(pupil[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline

            for i,Time in enumerate(dil_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date}
                df_list.append(new_dict)

        for j,trial in enumerate(trial_matrix):
            baseline = np.mean(dff[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
         
            for i,Time in enumerate(trial_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "dff","ID" : id, "Date": date}
                df_list.append(new_dict)

        for j,trial in enumerate(face_matrix):
            baseline = np.mean(face[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
          
            for i,Time in enumerate(face_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Face","ID" : id, "Date": date}
                df_list.append(new_dict)

lc_align_df = pd.DataFrame.from_dict(df_list)


In [ ]:
###onset aligned ACC trace
plt.figure(figsize = (2,1.2))
ax = sns.lineplot(data = acc_align_df.groupby(["ID","Time from Onset (s)","type"]).mean().reset_index(), x = "Time from Onset (s)", y = "value", hue = "type",errorbar = "se",palette= ["purple","dodgerblue","green"],legend= None,linewidth = 1)
format_ax(ax,(-1,0.5),(-0.5,1),0.5,0.5)
ax.set_xlabel("Time from dilation onset (s)")
ax.set_ylabel('Z-Score')
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/ACC_aligned.pdf",format ="pdf",transparent = True,bbox_inches = "tight")




In [ ]:
###onset aligned LC trace
plt.figure(figsize = (2,1.2))
ax = sns.lineplot(data = lc_align_df.groupby(["ID","Time from Onset (s)","type"]).mean().reset_index(), x = "Time from Onset (s)", y = "value", hue = "type",errorbar = "se",palette= ["purple","dodgerblue","green"],legend= None,linewidth = 1)
format_ax(ax,(-1,0.5),(-0.5,2),0.5,0.5)
ax.set_xlabel("Time from dilation onset (s)")
ax.set_ylabel('Z-Score')
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/LC_aligned.pdf",format ="pdf",transparent = True,bbox_inches = "tight")




In [ ]:
###hilbert trace for ACC


id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}


df_list = []
for id,dates in id_date.items():
    for date in dates:
       
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore(np.array(df["Running Speed"]))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])
        
        df = pd.DataFrame(pupil)
        df = df.rename(columns= {0:"pupil"})
        df["dff"] = dff
        df["face"] = face
        phases = np.angle(hilbert(df["pupil"]))
        df["phase_bin"] = pd.cut(phases,128,labels = False)
        df["ID"] = id
        df["Date"] = date
        df["Region"] = "ACC"
        df_list.append(df)
        
acc_hilbert_df = pd.concat(df_list)


In [ ]:
###hilbert trace for LC


id_date = {
  "LC01":["20240419"],
  "LC02":["20240416"],
  "LC03":["20240419"],
  "LC04":["20240416","20240417"],
  "LC05":["20240513"],
  "LC06":["20240513"],
}


df_list = []
for id,dates in id_date.items():
    for date in dates:
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/Nithik/ACC_Arousal_Paper/LC_data/{id}_{date}_fixed.csv"
        df = pd.read_csv(path)
        time = df["t"]
        pupil = np.array(zscore(df["pupil"])) ###zscore pupil data
        face = np.array(zscore(df["mot_whisk"]))###zscore face data
        dff = np.array(zscore(df["fp_lc"])) ###zscore dff data

        df = pd.DataFrame(pupil)
        df = df.rename(columns= {0:"pupil"})
        df["dff"] = dff
        df["face"] = face
        phases = np.angle(hilbert(df["pupil"]))
        df["phase_bin"] = pd.cut(phases,128,labels = False)
        df["ID"] = id
        df["Date"] = date
        df["Region"] = "LC"

        df_list.append(df)
  
lc_hilbert_df = pd.concat(df_list)

              
hilbert_df = pd.concat([acc_hilbert_df,lc_hilbert_df])
###label hilbert_df
def label_trial(row):
    if 32<= row['phase_bin']<=64:
      return 'Dilating'
    elif 64< row['phase_bin']<=96:
      return "Constricting"
      
hilbert_df['Pupil_State'] = hilbert_df.apply(label_trial, axis=1)



In [ ]:
hilbert_df

In [ ]:
ax = sns.lineplot(data = hilbert_df.groupby(["ID","phase_bin","Region","Pupil_State"]).mean().reset_index().query("Region == 'ACC'"), x = "phase_bin", y = "pupil", errorbar = "se",linewidth = 1,legend = None,color = "dodgerblue")


In [ ]:
###plot ACC hilbert over pupil

plt.figure(figsize = (2,1.2))
ax = sns.lineplot(data = hilbert_df.groupby(["ID","phase_bin","Region"]).mean().reset_index().query("Region == 'ACC'"), x = "phase_bin", y = "pupil", errorbar = "se",linewidth = 1,legend = None,color = "dodgerblue")
ax = sns.lineplot(data = hilbert_df.groupby(["ID","phase_bin","Region"]).mean().reset_index().query("Region == 'ACC'"), x = "phase_bin", y = "dff", color = "green",errorbar = "se",linewidth = 1)

format_ax(ax,(0,128),(-1,2),32,0.5)
ax.set_xlabel("Pupil phase")
ax.set_ylabel('Z-Score')
labels = ["-π","-π/2","0","π/2","π"]
ax.set_xticklabels(labels)


#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/ACC_hilbert.pdf",format ="pdf",transparent = True,bbox_inches = "tight")





In [ ]:
###plot LC hilbert over pupil

plt.figure(figsize = (2,1.2))
ax = sns.lineplot(data = hilbert_df.groupby(["ID","phase_bin","Region"]).mean().reset_index().query("Region == 'LC'"), x = "phase_bin", y = "pupil", color = "dodgerblue",errorbar = "se",linewidth = 1)
ax = sns.lineplot(data = hilbert_df.groupby(["ID","phase_bin","Region"]).mean().reset_index().query("Region == 'LC'"), x = "phase_bin", y = "dff", color = "green",errorbar = "se",linewidth = 1)

format_ax(ax,(0,128),(-1,2),32,0.5)
ax.set_xlabel("Pupil phase")
ax.set_ylabel('Z-Score')
ax.set_xticklabels(labels)



plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/LC_hilbert.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###plot LC dff over ACC dff

plt.figure(figsize = (2,1.2))
ax = sns.lineplot(data = hilbert_df.groupby(["ID","phase_bin","Region"]).mean().reset_index(), x = "phase_bin", y = "dff", hue = "Region",errorbar = "se",linewidth = 1,legend = None,palette = ["deeppink","gold"])

format_ax(ax,(0,128),(-1,1.5),32,0.5)
ax.set_xlabel("Pupil phase")
ax.set_ylabel('∆ F/F')
ax.axvspan(32,64,alpha = 0.1,color = "red")
ax.axvspan(64,96,alpha = 0.1,color = "blue")
ax.set_xticklabels(labels)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/LCvsACC_hilbert.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###plot LC vs ACC during dilating phase
plt.figure(figsize = (0.6,1.2))
g = sns.barplot(data = hilbert_df.groupby(["ID",'Pupil_State',"Region"]).mean().reset_index().query("Pupil_State == 'Dilating'"), x = "Region", y = "dff",errorbar="se",fill = None,errwidth= 1,edgecolor = "red")
sns.stripplot(data = hilbert_df.groupby(["ID",'Pupil_State',"Region"]).mean().reset_index().query("Pupil_State == 'Dilating'"), x = "Region", y = "dff",color = "black",alpha = 0.3,s = 4,jitter = 0.1)
g.xaxis.label.set_visible(False)
g.set_yticks(np.arange(0,1.4,0.4))
g.set(ylim=(0, 1.2))
g.set_ylabel('∆ F/F (z-scr)')
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/dilating_dff.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###LC vs ACC during dilating phase stats
stat_df = hilbert_df.groupby(["ID",'Pupil_State',"Region"]).mean().reset_index().query("Pupil_State == 'Constricting'")
group1 = stat_df[stat_df['Region']=="LC"]
group2 = stat_df[stat_df['Region']=="ACC"]
mannwhitneyu(group1["dff"], group2["dff"],nan_policy= "omit")

In [ ]:
###plot LC vs ACC during constricting phase
plt.figure(figsize = (0.6,1.2))
g = sns.barplot(data = hilbert_df.groupby(["ID",'Pupil_State',"Region"]).mean().reset_index().query("Pupil_State == 'Constricting'"), x = "Region", y = "dff",errorbar="se",fill = None,errwidth= 1,edgecolor = "blue")
sns.stripplot(data = hilbert_df.groupby(["ID",'Pupil_State',"Region"]).mean().reset_index().query("Pupil_State == 'Constricting'"), x = "Region", y = "dff",color = "black",alpha = 0.3,s = 4,jitter = 0.1)
g.xaxis.label.set_visible(False)
g.set_yticks(np.arange(-0.6,1,0.3))
g.set(ylim=(-0.6, 0.6))
g.set_ylabel('∆ F/F (z-scr)')
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/constricting_dff.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###LC vs ACC during constricting phase stats
stat_df = hilbert_df.groupby(["ID",'Pupil_State',"Region"]).mean().reset_index().query("Pupil_State == 'Constricting'")
group1 = stat_df[stat_df['Region']=="LC"]
group2 = stat_df[stat_df['Region']=="ACC"]
mannwhitneyu(group1["dff"], group2["dff"],nan_policy= "omit")

In [ ]:
###compute dilating to constrict dff ratios
constrict_dff = hilbert_df.groupby(["ID",'Pupil_State',"Region"]).mean().reset_index().query("Pupil_State == 'Constricting'").reset_index()["dff"]
dilate_dff = hilbert_df.groupby(["ID",'Pupil_State',"Region"]).mean().reset_index().query("Pupil_State == 'Dilating'").reset_index()["dff"]
ratios = constrict_dff/dilate_dff
IDs = hilbert_df.groupby(["ID",'Pupil_State',"Region"]).mean().reset_index().query("Pupil_State == 'Constricting'").reset_index()["ID"]
Regions = hilbert_df.groupby(["ID",'Pupil_State',"Region"]).mean().reset_index().query("Pupil_State == 'Constricting'").reset_index()["Region"]
ratio_df = pd.DataFrame(IDs)
ratio_df = ratio_df.rename(columns= {0:"ID"})
ratio_df["Region"] = Regions
ratio_df["ratio"] = ratios

In [ ]:
###plot LC vs ACC ratios
plt.figure(figsize = (0.6,1.2))
g = sns.barplot(data = ratio_df, x = "Region", y = "ratio",errorbar="se",fill = None,errwidth= 1)
sns.stripplot(data = ratio_df, x = "Region", y = "ratio",color = "black",alpha = 0.3,s = 4,jitter = 0.1)
g.xaxis.label.set_visible(False)
g.set_yticks(np.arange(-0.9,0.9,0.3))
g.set(ylim=(-0.9, 0.6))
g.set_ylabel('Ratio')
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/LCvsACC_hilbert_ratio.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
ratio_df

In [ ]:
###LC vs ACC during constricting phase stats
stat_df = ratio_df
group1 = ratio_df[ratio_df['Region']=="LC"]
group2 = ratio_df[ratio_df['Region']=="ACC"]
mannwhitneyu(group1["ratio"], group2["ratio"],nan_policy= "omit")

In [ ]:
test = hilbert_df.groupby(["ID",'Pupil_State',"Region"]).mean().reset_index()
scatter_df = test.pivot(index='ID', columns='Pupil_State', values='dff').reset_index()
scatter_df["Region"] = Regions

In [ ]:
scatter_df

In [ ]:
plt.figure(figsize = (1.2,1.2))
ax = sns.scatterplot(data = scatter_df, x = "Constricting", y = "Dilating",hue = "Region",size = 1,legend = None,palette = ["deeppink","gold"])
format_ax(ax,(-0.5,0),(0,0.5),0.25,0.25)
#ax.plot([-0.5,0], [0,0.5], linewidth=1,linestyle = "dashed",color = "black")
ax.set_ylabel('Dilating ∆ F/F')
ax.set_xlabel('Constricting ∆ F/F')



In [ ]:
###LC vs ACC Q4/Q1 ratio

###get ACC quartile aligned bar df

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
 }
labels = ["1","2","3","4"]
df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_acc_events(id,date)
        peak_ix = events["peak_ix"]
        onsets = [on for on in events["onsets"]] ###get pupil onsets ix
        onsets_ix = events["onsets"]
        offsets = [off for off in events["offsets"]] ###get pupil offsets ix
        amplitudes = events["amplitudes"]
        binned_amps = np.array(pd.qcut(amplitudes,q = 4,labels = labels))
     

  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

       

     
        for i,(on,off) in enumerate(zip(onsets,offsets)):
          baseline = np.mean(dff[on - 40:on - 10])
          pre_dff =np.mean(dff[on-5:on]) - baseline
          during_dff = np.mean(dff[on:off]) - baseline

          trial_dff = dff[on-10:off]
          trial_dff = trial_dff - np.min(trial_dff)

          ###calculate T50
          max_ix = np.argmax(trial_dff)
          peak_dff =  np.max(trial_dff)

          target = peak_dff/2
          t50_ix = np.where(trial_dff > target)[0][0]
          t50 = t50_ix/20





          
          


          new_dict = { "trial" : i,"ID" : id, "Date": date,"amp":amplitudes[i],"animal_amp_bin":binned_amps[i],"Pre Onset ∆ F/F":pre_dff,"∆ F/F": during_dff,"Half Max (s)": half_max,"T50 (s)": t50}
          df_list.append(new_dict)


bar_df = pd.DataFrame.from_dict(df_list)
q1 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '1'").reset_index()["∆ F/F"]
q4 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '4'").reset_index()["∆ F/F"]
ID = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '1'").reset_index()["ID"]
dff_ratios = q4/q1
acc_ratio = pd.DataFrame(ID)
acc_ratio = acc_ratio.rename(columns= {0:"ID"})
acc_ratio["ratio"] = dff_ratios
acc_ratio["Region"] = "ACC"



In [ ]:
plt.figure(figsize = (1,1.25))
g = sns.barplot(data = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "Half Max (s)",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "Half Max (s)",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Dilation Amplitude\n Quartile")
g.set_yticks(np.arange(0,12,2))
g.set(ylim=(0,10))
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_ampbin_dff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
plt.figure(figsize = (1,1.25))
g = sns.barplot(data = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "T50 (s)",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "T50 (s)",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Dilation Amplitude\n Quartile")
# g.set_yticks(np.arange(0,30,2))
# g.set(ylim=(0,30))
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_ampbin_dff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###LC vs ACC Q4/Q1 ratio

###get LC quartile aligned bar df

id_date = {
  "LC01":["20240419"],
  "LC02":["20240416"],
  "LC03":["20240419"],
  "LC04":["20240416","20240417"],
}
labels = ["1","2","3","4"]
df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_lc_events(id,date)
        peak_ix = events["peak_ix"]
        onsets = [on for on in events["onsets"]] ###get pupil onsets ix
        onsets_ix = events["onsets"]
        offsets = [off for off in events["offsets"]] ###get pupil offsets ix
        amplitudes = events["amplitudes"]
        binned_amps = np.array(pd.qcut(amplitudes,q = 4,labels = labels))
     

  
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/Nithik/ACC_Arousal_Paper/LC_data/{id}_{date}.csv"
        df = pd.read_csv(path)
        time = df["t"]
        pupil = np.array(zscore(df["pupil"])) ###zscore pupil data
        face = np.array(zscore(df["mot_whisk"]))###zscore face data
        dff = np.array(zscore(df["fp_lc"])) ###zscore dff data


       

     
        for i,(on,off) in enumerate(zip(onsets,offsets)):
          baseline = np.mean(dff[on - 40:on - 10])
          pre_dff =np.mean(dff[on-5:on]) - baseline
          during_dff = np.mean(dff[on:off]) - baseline
       
          trial_dff = dff[on-20:off]
          trial_dff = trial_dff - np.min(trial_dff)
          #trial_dff = np.convolve(trial_dff, np.ones(10)/10, mode='same')
          # ###calculate half-max
          # max_ix = np.argmax(trial_dff)

          # peak_dff =  np.max(trial_dff) - np.min(trial_dff)
          # target = peak_dff/2
          #half_ix = np.where(trial_dff>target)[0][0]
          # half_max = half_ix/20

          ###calculate T50
          max_ix = np.argmax(trial_dff)
          peak_dff =  np.max(trial_dff) - np.min(trial_dff)

          target = peak_dff/2
          t50_ix = np.where(trial_dff > target)[0][0]
          t50 = t50_ix/20


          new_dict = { "trial" : i,"ID" : id, "Date": date,"amp":amplitudes[i],"animal_amp_bin":binned_amps[i],"Pre Onset ∆ F/F":pre_dff,"∆ F/F": during_dff,"Half Max (s)": half_max,"T50 (s)":t50}
          df_list.append(new_dict)


bar_df = pd.DataFrame.from_dict(df_list)
q1 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '1'").reset_index()["∆ F/F"]
q4 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '4'").reset_index()["∆ F/F"]
ID = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '1'").reset_index()["ID"]
dff_ratios = q4/q1
lc_ratio = pd.DataFrame(ID)
lc_ratio = lc_ratio.rename(columns= {0:"ID"})
lc_ratio["ratio"] = dff_ratios
lc_ratio["Region"] = "LC"




In [ ]:
plt.figure(figsize = (1,1.25))
g = sns.barplot(data = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "Half Max (s)",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "Half Max (s)",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Dilation Amplitude\n Quartile")
g.set_yticks(np.arange(0,12,2))
g.set(ylim=(0,10))
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_ampbin_dff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
plt.figure(figsize = (1,1.25))
g = sns.barplot(data = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "T50 (s)",errorbar = "se",fill = None,errwidth= 1)
sns.lineplot(data = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "T50 (s)",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Dilation Amplitude\n Quartile")
# g.set_yticks(np.arange(0,30,2))
# g.set(ylim=(0,30))
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_ampbin_dff_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
q_ratio_df = pd.concat([lc_ratio,acc_ratio])

In [ ]:
###plot LC vs ACC ratios
plt.figure(figsize = (0.8,1.2))
g = sns.barplot(data = q_ratio_df, x = "Region", y = "ratio",errorbar="se",fill = None,errwidth= 1)
sns.stripplot(data = q_ratio_df, x = "Region", y = "ratio",color = "black",alpha = 0.3,s = 4,jitter = 0.1)
g.xaxis.label.set_visible(False)
g.set_yticks(np.arange(0,30,5))
g.set(ylim=(0, 25))
g.set_ylabel('Ratio')
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/LCvsACC_q_ratio.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###LC vs ACC face-dff correlation stats
stat_df = q_ratio_df
group1 = stat_df[stat_df['Region']=="LC"]
group2 = stat_df[stat_df['Region']=="ACC"]
ttest_ind(group1["ratio"], group2["ratio"],nan_policy= "omit")

In [ ]:
###try t50 and t5 with averaged traces

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_acc_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        onsets_ix = events["onsets"]
  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore(np.array(df["Running Speed"]))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = 2, post = 10 )
        trial_matrix,trial_times = trial_align(onsets,time,dff,fps = 20,pre = 2, post = 10)
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = 2, post = 10)

        dff_trial = np.mean(trial_matrix,0)
        dff_trial = dff_trial - np.min(dff_trial)
        max_ix = np.argmax(dff_trial)
        min_ix = np.argmin(dff_trial)
        peak_dff =  np.max(dff_trial)
        target = peak_dff/2
        t50_ix = np.where(dff_trial[min_ix:] > target)[0][0] + min_ix
       # t50 = (t50_ix - min_ix)/20 ###relative to minimum
        t50 = (t50_ix - 40)/20 ###relative to dilation onset

  

        ###find onset of dff by finding continuous region from min to max

        regions = list(groupSequence(dff_trial[min_ix:max_ix]))
        corrected_regions = [region for region in regions if len(region) >=5 ]
        first_region = corrected_regions[0]
        region_ix = [np.where(dff_trial[min_ix:max_ix] == val)[0][0] + min_ix for val in first_region]
        
        onset_ix,offset_ix = region_ix[0],region_ix[-1]
        onset_time = (onset_ix-40)/20


                
        new_dict = {"ID" : id, "Date": date,"T50 (s)": t50,"Region":"ACC","Onset":onset_time}
        df_list.append(new_dict)
        




        # ax = plt.gca()
        # plt.plot(dff_trial)
        # ax.axvline(40)
        # ax.axvspan(onset_ix,offset_ix,alpha = 0.2)
        # #ax.axhline(target)

        # plt.scatter(max_ix,peak_dff,color = "blue")
        # plt.scatter(t5_ix,dff_trial[t5_ix],color = "red")

        # plt.show()
        # plt.clf()
    

      
acc_t50_df = pd.DataFrame.from_dict(df_list)


In [ ]:
lc_bar_

In [ ]:
###try t50 and t5 with averaged traces for LC


id_date = {
  "LC01":["20240419"],
  "LC02":["20240416"],
  "LC03":["20240419"],
  "LC04":["20240416","20240417"],
  "LC05":["20240513"],
  "LC06":["20240513"],
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_lc_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        onsets_ix = events["onsets"]
  
       
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/Nithik/ACC_Arousal_Paper/LC_data/{id}_{date}_fixed.csv"
        df = pd.read_csv(path)
        time = df["t"]
        pupil = np.array(zscore(df["pupil"])) ###zscore pupil data
        face = np.array(zscore(df["mot_whisk"]))###zscore face data
        dff = np.array(zscore(df["fp_lc"])) ###zscore dff data

        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = 2, post = 10 )
        trial_matrix,trial_times = trial_align(onsets,time,dff,fps = 20,pre = 2, post = 10)
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = 2, post = 10)

        dff_trial = np.nanmean(trial_matrix,0)
        dff_trial = dff_trial - np.min(dff_trial)
        max_ix = np.argmax(dff_trial)
        min_ix = np.argmin(dff_trial)
        peak_dff =  np.max(dff_trial)
        target = peak_dff/2
        t50_ix = np.where(dff_trial[min_ix:] > target)[0][0] + min_ix
        t50 = (t50_ix - 40)/20 ###relative to dilation onset

   
      
        ###find onset of dff by finding continuous region from min to max

        regions = list(groupSequence(dff_trial[min_ix:max_ix]))
        corrected_regions = [region for region in regions if len(region) >=5 ]
        first_region = corrected_regions[0]
        region_ix = [np.where(dff_trial[min_ix:max_ix] == val)[0][0] + min_ix for val in first_region]
        
        onset_ix,offset_ix = region_ix[0],region_ix[-1]
        onset_time = (onset_ix-40)/20

                
        new_dict = {"ID" : id, "Date": date,"T50 (s)": t50,"Region":"LC","Onset":onset_time}
        df_list.append(new_dict)




        # ax = plt.gca()
        # plt.plot(dff_trial)
        # ax.axvline(40)
        # ax.axvspan(onset_ix,offset_ix,alpha = 0.2)

        # #ax.axhline(target)

        # plt.scatter(max_ix,peak_dff,color = "blue")
        # plt.scatter(min_ix,dff_trial[min_ix],color = "black")

        # plt.scatter(t50_ix,dff_trial[t50_ix],color = "red")

        # plt.show()
        # plt.clf()
    

      
lc_t50_df = pd.DataFrame.from_dict(df_list)
t50_df = pd.concat([lc_t50_df,acc_t50_df])

In [ ]:
t50_df

In [ ]:
###LC vs ACC t50
plt.figure(figsize = (0.6,1.2))
g = sns.barplot(data = t50_df.groupby(["ID","Region"]).mean().reset_index(), x = "Region", y = "T50 (s)",errorbar="se",fill = None,errwidth= 1,order = ["ACC","LC"])
sns.stripplot(data = t50_df.groupby(["ID","Region"]).mean().reset_index(), x = "Region", y = "T50 (s)",color = "black",alpha = 0.3,s = 4,jitter = 0.1,order = ["ACC","LC"])
g.xaxis.label.set_visible(False)
g.set_yticks(np.arange(-1,0.5,.5))
g.set(ylim=(-1, 0))
g.set_ylabel("T50 (s)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/t50.pdf",format ="pdf",transparent = True,bbox_inches = "tight")

In [ ]:
###LC vs ACC activity onset
plt.figure(figsize = (0.6,1.2))
g = sns.barplot(data = t50_df.groupby(["ID","Region"]).mean().reset_index(), x = "Region", y = "Onset",errorbar="se",fill = None,errwidth= 1,order =["ACC","LC"] )
sns.stripplot(data = t50_df.groupby(["ID","Region"]).mean().reset_index(), x = "Region", y = "Onset",color = "black",alpha = 0.3,s = 4,jitter = 0.1,order = ["ACC","LC"])
g.xaxis.label.set_visible(False)
g.set_yticks(np.arange(-1.5,0.5,.5))
g.set(ylim=(-1.5, 0))
g.set_ylabel("Activity Onset (s)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/activity_onset.pdf",format ="pdf",transparent = True,bbox_inches = "tight")

In [ ]:
###LC vs ACC face-dff correlation stats
stat_df = t50_df.groupby(["ID","Region"]).mean().reset_index()
group1 = stat_df[stat_df['Region']=="LC"]
group2 = stat_df[stat_df['Region']=="ACC"]
mannwhitneyu(group1["T50 (s)"], group2["T50 (s)"],nan_policy= "omit")

In [ ]:
t50_df.groupby(["ID","Region"]).mean().reset_index()

In [ ]:
###LC vs ACC activity onset stats
stat_df = t50_df.groupby(["ID","Region"]).mean().reset_index()
group1 = stat_df[stat_df['Region']=="LC"]
group2 = stat_df[stat_df['Region']=="ACC"]
mannwhitneyu(group1["Onset"], group2["Onset"],nan_policy= "omit")

In [ ]:
###LC vs ACC onset dff - peak dff


id_date = {
  "LC01":["20240419"],
  "LC02":["20240416"],
  "LC03":["20240419"],
  "LC04":["20240416","20240417"],
  "LC05":["20240513"],
  "LC06":["20240513"],
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_lc_events(id,date)
        peak_ix = events["peak_ix"]
        onsets = [on for on in events["onsets"]] ###get pupil onsets ix
        onsets_ix = events["onsets"]
        offsets = [off for off in events["offsets"]] ###get pupil offsets ix
        amplitudes = events["amplitudes"]
     
     

  
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/Nithik/ACC_Arousal_Paper/LC_data/{id}_{date}_fixed.csv"
        df = pd.read_csv(path)
        time = df["t"]
        pupil = np.array(zscore(df["pupil"])) ###zscore pupil data
        face = np.array(zscore(df["mot_whisk"]))###zscore face data
        dff = np.array(zscore(df["fp_lc"])) ###zscore dff data

       

     
        for i,(on,peak) in enumerate(zip(onsets_ix,peak_ix)):
            delta = dff[peak] - dff[on]
 


            new_dict = { "trial" : i,"ID" : id, "Date": date,"Peak-Onset":delta,"Region":"LC"}
            df_list.append(new_dict)


lc_bar_df = pd.DataFrame.from_dict(df_list)


In [ ]:
lc_bar_df

In [ ]:
###LC vs ACC onset dff - peak dff



id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_acc_events(id,date)
        peak_ix = events["peak_ix"]
        onsets = [on for on in events["onsets"]] ###get pupil onsets ix
        onsets_ix = events["onsets"]
        offsets = [off for off in events["offsets"]] ###get pupil offsets ix
        amplitudes = events["amplitudes"]
     
     

  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore(np.array(df["Running Speed"]))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

       

     
        for i,(on,peak) in enumerate(zip(onsets_ix,peak_ix)):
            delta = dff[peak] - dff[on]
 


            new_dict = { "trial" : i,"ID" : id, "Date": date,"Peak-Onset":delta,"Region":"ACC"}
            df_list.append(new_dict)


acc_bar_df = pd.DataFrame.from_dict(df_list)
bar_df = pd.concat([lc_bar_df,acc_bar_df])

In [ ]:
###LC vs ACC face-dff correlation
plt.figure(figsize = (0.6,1.2))
g = sns.barplot(data = bar_df.groupby(["ID","Region"]).mean().reset_index(), x = "Region", y = "Peak-Onset",errorbar="se",fill = None,errwidth= 1)
sns.stripplot(data = bar_df.groupby(["ID","Region"]).mean().reset_index(), x = "Region", y = "Peak-Onset",color = "black",alpha = 0.3,s = 4,jitter = 0.1)
g.xaxis.label.set_visible(False)
g.set_yticks(np.arange(-2,1,0.5))
g.set(ylim=(-2, 0.5))
g.set_ylabel("Peak - Onset (∆ F/F)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/peak-onset.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###LC vs ACC face-dff correlation stats
stat_df = bar_df
group1 = stat_df[stat_df['Region']=="LC"]
group2 = stat_df[stat_df['Region']=="ACC"]
mannwhitneyu(group1["Peak-Onset"], group2["Peak-Onset"],nan_policy= "omit")